# Fit Neg Binomial model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial copula until convergence
    - Fix $\beta$ and variance compoenent parameters, fit $r$ using Newton
    - Repeat until convergence
    
# No block updates (only initialize $r$)

In [1]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 10
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [2.0, 1.0, 2.0, 6.0, 1.0]
 [8.0, 5.0, 3.0, 5.0, 2.0]
 [2.0, 4.0, 4.0, 10.0, 2.0]
 [4.0, 3.0, 7.0, 4.0, 8.0]
 [10.0, 10.0, 9.0, 1.0, 1.0]
 [4.0, 4.0, 3.0, 4.0, 3.0]
 [9.0, 6.0, 5.0, 4.0, 4.0]
 [6.0, 1.0, 3.0, 6.0, 0.0]
 [4.0, 7.0, 3.0, 3.0, 3.0]
 [5.0, 5.0, 3.0, 6.0, 2.0]
 [7.0, 6.0, 5.0, 2.0, 3.0]
 [3.0, 2.0, 7.0, 1.0, 4.0]
 [2.0, 3.0, 2.0, 2.0, 6.0]
 ⋮
 [2.0, 3.0, 1.0, 4.0, 2.0]
 [1.0, 3.0, 6.0, 2.0, 2.0]
 [2.0, 1.0, 3.0, 5.0, 1.0]
 [2.0, 3.0, 1.0, 1.0, 5.0]
 [2.0, 3.0, 4.0, 4.0, 10.0]
 [2.0, 3.0, 6.0, 2.0, 6.0]
 [2.0, 3.0, 1.0, 8.0, 2.0]
 [8.0, 0.0, 6.0, 3.0, 5.0]
 [5.0, 3.0, 2.0, 3.0, 6.0]
 [5.0, 6.0, 6.0, 9.0, 5.0]
 [0.0, 2.0, 1.0, 4.0, 3.0]
 [3.0, 6.0, 1.0, 3.0, 3.0]

In [2]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [3]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

Initializing NegBin r to Poisson regression values
initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -124348.27381045274 39999
2 -124348.27381045274 -124348.27381045274 39999
initializing variance components using MM-Algorithm
gcm.Σ = [220.41649889527125, 1469.9374107479462]
gcm.θ = [-7.661495143796714, 219.50634778984383, 1476.6337551112845]
gcm.θ = [1.505897960765723, 220.41649889527125, 1469.9374107479462]
gcm.θ = [1.505897960765723, 220.41649889527125, 1469.9374107479462]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more

1-element Vector{Float64}:
 5.567431570219433

In [4]:
β_true

1.4552872326068422

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [5]:
GLMCopula.loglikelihood!(gcm, true, true)

-119242.84209292504

## fitting routine

In [6]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100,
#     tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",
#     mehrotra_algorithm="yes", warm_start_init_point="yes",
    hessian_approximation = "limited-memory"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

-119215.76591601499

In [7]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.4983340299072807; true β = 1.4552872326068422
estimated variance component 1 = 0.026531702645107527; true variance component 1 = 0.1
estimated variance component 2 = 0.0; true variance component 2 = 0.5
estimated r = 5.567431570219433; true r = 10


# Block updates

In [ ]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 10
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

In [ ]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [ ]:
initialize_model!(gcm)
@show gcm.β
# @show gcm.Σ
@show gcm.r

In [ ]:
β_true

In [ ]:
GLMCopula.loglikelihood!(gcm, true, true)

In [ ]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100,
#     tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",
#     mehrotra_algorithm="yes", warm_start_init_point="yes",
    hessian_approximation = "limited-memory"))

In [ ]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");